In [1]:
import argparse
from ray.tune import Analysis
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy 
import pdb 

ModuleNotFoundError: No module named 'ray'

In [ ]:
logdir = '/data/petabyte/safe_autonomy/ieee_aero_2022/experiments/docking_2d/expr_20210914_005226'

In [ ]:
all_subdirs = next(os.walk(logdir))[1]

In [ ]:
all_subdirs

In [ ]:
# preprocessing logic 

# keep track of all csv files
csv_file_tracker = []
for d in all_subdirs: 
    if d == 'training_logs':
        continue
    else: 
        csv_path = logdir + '/' + d + '/' + 'progress.csv'
        csv_file_tracker.append(csv_path)


In [ ]:
data_dfs = [pd.read_csv(csv_file_tracker[i]) for i in range(len(csv_file_tracker))]

Make graphs and visuals of all the data present - use the multiple loop approach from earlier notebook 

In [ ]:
plt.figure()
plt.title('Steps vs Success Mean')
plt.ylabel('Success Mean')
plt.xlabel('Number of steps')

seed = 0
for d in all_subdirs: 
    if d == 'training_logs':
        continue
    else: 
        seed_name = 'seed_' + str(seed)
        csv_path = logdir + '/' + d + '/' + 'progress.csv'
        data = pd.read_csv(csv_path)
        steps = data['timesteps_total']
        success_mean= data['custom_metrics/outcome/success_mean']
        plt.plot(steps,success_mean,label=seed_name)
        seed = seed + 1
plt.legend()


In [ ]:
plt.figure()
plt.title('Steps vs Episode Length Mean')
plt.ylabel('Episode Length Mean')
plt.xlabel('Number of steps')

seed = 0
for d in all_subdirs: 
    if d == 'training_logs':
        continue
    else: 
        seed_name = 'seed_' + str(seed)
        csv_path = logdir + '/' + d + '/' + 'progress.csv'
        data = pd.read_csv(csv_path)
        steps = data['timesteps_total']
        success_mean= data['episode_len_mean']
        plt.plot(steps,success_mean,label=seed_name)
        seed = seed + 1

plt.legend()


In [ ]:
plt.figure()
plt.title('Steps vs Reward')
plt.ylabel('Reward')
plt.xlabel('Number of steps')

seed = 0
for d in all_subdirs: 
    if d == 'training_logs':
        continue
    else: 
        seed_name = 'seed_' + str(seed)
        csv_path = logdir + '/' + d + '/' + 'progress.csv'
        data = pd.read_csv(csv_path)
        steps = data['timesteps_total']
        success_mean= data['episode_reward_mean']
        plt.plot(steps,success_mean,label=seed_name)
        seed = seed + 1

plt.legend()


View DataFrames to find dfs that end around 4*10^6 steps

In [ ]:
data_dfs[1]

In [ ]:
data_dfs[3]

In [ ]:
data_dfs[8]

In [ ]:
data_dfs[8].columns

In [ ]:
# find the set of unique x values i.e. timestep data 
key_timesteps = 'timesteps_total'
timesteps_total_track = []
for ds in data_dfs: 
    for i in range(ds.shape[0]):                
        timestep_total = ds.iloc[[i]][key_timesteps][i]
        timesteps_total_track.append(timestep_total)

timesteps = np.array(timesteps_total_track)
unique_timesteps = np.unique(timesteps) 

In [ ]:
#view intermediate results 
print(unique_timesteps)
print(unique_timesteps.max())
print(unique_timesteps.min())
print("range:",unique_timesteps.max() - unique_timesteps.min())

In [ ]:
# interpolate each experiment's data over that set of unique x values 

In [ ]:
clipped_timesteps = data_dfs[8]['timesteps_total']

In [ ]:
from scipy import interpolate
sns.set_theme()

In [ ]:
key_timesteps = 'timesteps_total'
key_eps_len_mean = 'episode_len_mean'
key_success_mean = 'custom_metrics/outcome/success_mean'
key_eps_reward_mean = 'episode_reward_mean'


In [ ]:
timesteps_total_track = []
episode_len_mean_track = []
success_mean_track = []
eps_reward_mean_track = []

# dont need to walk through step by step , can just grab columns 
for ds in data_dfs:                 
    timestep_total = ds[key_timesteps]
    episode_len_mean = ds[key_eps_len_mean]
    success_mean = ds[key_success_mean]
    reward_mean = ds[key_eps_reward_mean]
        
    # interpolation functions
    func_time_v_eps_len = interpolate.interp1d(timestep_total,episode_len_mean,fill_value='extrapolate')
    func_time_v_success = interpolate.interp1d(timestep_total,success_mean,fill_value='extrapolate')
    func_time_v_reward = interpolate.interp1d(timestep_total,reward_mean,fill_value='extrapolate')
    interp_eps_len = func_time_v_eps_len(clipped_timesteps)
    interp_success = func_time_v_success(clipped_timesteps)
    interp_reward = func_time_v_reward(clipped_timesteps)
        
    for i in clipped_timesteps:
        timesteps_total_track.append(i)
    for i in interp_eps_len: 
        episode_len_mean_track.append(i)
    for i in interp_success: 
        success_mean_track.append(i)
    for i in interp_reward: 
        eps_reward_mean_track.append(i)


In [ ]:
sns.set_theme()
sns.set(font_scale=1.5)
timesteps_total_v_episode_len_mean = pd.DataFrame()
timesteps_total_v_episode_len_mean[key_timesteps] = timesteps_total_track
timesteps_total_v_episode_len_mean[key_eps_len_mean] = episode_len_mean_track

In [ ]:
episode_mean_len_plot = sns.relplot(data=timesteps_total_v_episode_len_mean,x='timesteps_total',y='episode_len_mean',kind='line')
episode_mean_len_plot.set_axis_labels("Timesteps","Episode Length")


In [ ]:
timesteps_total_v_success_mean = pd.DataFrame()
timesteps_total_v_success_mean[key_timesteps] = timesteps_total_track
timesteps_total_v_success_mean['success_mean'] = success_mean_track

In [ ]:
success_mean_plot = sns.relplot(data=timesteps_total_v_success_mean,x='timesteps_total',y='success_mean',kind='line')
success_mean_plot.set_axis_labels("Timesteps","Success Rate") 

In [ ]:
timesteps_total_v_episode_reward_mean = pd.DataFrame()
timesteps_total_v_episode_reward_mean[key_timesteps] = timesteps_total_track
timesteps_total_v_episode_reward_mean['episode_reward_mean'] = eps_reward_mean_track

In [ ]:
reward_plot = sns.relplot(data=timesteps_total_v_episode_reward_mean,x='timesteps_total',y='episode_reward_mean',kind='line')
reward_plot.set_axis_labels("Timesteps","Average Return")

In [ ]:
episode_mean_len_plot.savefig('docking2d_eps_len_plot.png',dpi=1200)
success_mean_plot.savefig('docking2d_success_mean.png',dpi=1200)
reward_plot.savefig('docking2d_reward_graph.png',dpi=1200)

## Constraint Violation Ratio Mean Graph

In [ ]:
key_timesteps = 'timesteps_total'
key_const_viol = 'custom_metrics/constraint_violation.max_vel_constraint.ratio_mean'


In [ ]:
plt.figure()
plt.title('Steps vs Ratio Mean Constraint Violation')
plt.ylabel('Ration Mean Constraint Violation')
plt.xlabel('Number of steps')

seed = 0
for d in all_subdirs: 
    if d == 'training_logs':
        continue
    else: 
        seed_name = 'seed_' + str(seed)
        csv_path = logdir + '/' + d + '/' + 'progress.csv'
        data = pd.read_csv(csv_path)
        steps = data['timesteps_total']
        success_mean= data[key_const_viol]
        plt.plot(steps,success_mean,label=seed_name)
        seed = seed + 1

plt.legend()


In [ ]:

timesteps_total_track = []
const_viol_ratio_track = []

# dont need to walk through step by step , can just grab columns 
for ds in data_dfs: 
        timestep_total = ds[key_timesteps]
        const_viol_ratio = ds[key_const_viol]
        # interpolation functions
        func_time_v_constraint = interpolate.interp1d(timestep_total,const_viol_ratio,fill_value='extrapolate')
        interp_eps_len = func_time_v_constraint(clipped_timesteps)
        
        for i in clipped_timesteps:
            timesteps_total_track.append(i)
        for i in const_viol_ratio: 
            const_viol_ratio_track.append(i)

In [ ]:
timesteps_total_v_constr_viol = pd.DataFrame()
timesteps_total_v_constr_viol[key_timesteps] = timesteps_total_track
timesteps_total_v_constr_viol['constr_viol'] = const_viol_ratio_track

In [ ]:
sns.set_theme()
sns.set(font_scale=1.5)
constr_viol_plot = sns.relplot(data=timesteps_total_v_constr_viol,x='timesteps_total',y='constr_viol',kind='line')
constr_viol_plot.set_axis_labels("Timesteps","Constraint Violation Ratio")

In [ ]:
constr_viol_plot.savefig('docking2d_constraint_violation.png',dpi=1200)

## Graph - delta v vs timesteps

In [ ]:
data_dfs[0].columns

In [ ]:
key_delta_v = 'custom_metrics/delta_v_total_mean'

In [ ]:
plt.figure()
plt.title('Steps vs Delta V')
plt.ylabel('Delta V ')
plt.xlabel('Number of steps')

seed = 0
for d in all_subdirs: 
    if d == 'training_logs':
        continue
    else: 
        seed_name = 'seed_' + str(seed)
        csv_path = logdir + '/' + d + '/' + 'progress.csv'
        data = pd.read_csv(csv_path)
        steps = data['timesteps_total']
        success_mean= data[key_delta_v]
        plt.plot(steps,success_mean,label=seed_name)
        seed = seed + 1

plt.legend()


In [ ]:
data_dfs[0][key_delta_v].min()

In [ ]:
data_dfs[0][key_delta_v].max()

In [ ]:
clipped_timesteps = data_dfs[8]['timesteps_total']
timesteps_total_track = []
delta_v_track = []

# dont need to walk through step by step , can just grab columns 
for ds in data_dfs: 
        timestep_total = ds[key_timesteps]
        delta_v_mean = ds[key_delta_v]
        # interpolation functions
        func_time_v_constraint = interpolate.interp1d(timestep_total,delta_v_mean,fill_value='extrapolate')
        interp_delta_v = func_time_v_constraint(clipped_timesteps)
        
        for i in clipped_timesteps:
            timesteps_total_track.append(i)
        for i in interp_delta_v: 
            delta_v_track.append(i)

In [ ]:
timesteps_total_v_constr_viol = pd.DataFrame()
timesteps_total_v_constr_viol[key_timesteps] = timesteps_total_track
timesteps_total_v_constr_viol['delta_v'] = delta_v_track

In [ ]:
sns.set_theme()
sns.set(font_scale=1.5)
deltav_plot = sns.relplot(data=timesteps_total_v_constr_viol,x='timesteps_total',y='delta_v',kind='line')
deltav_plot.set_axis_labels("Timesteps","Delta V")

In [ ]:
deltav_plot.savefig('docking2d_delta_v.png',dpi=1200)